In [2]:
from IPython.display import clear_output
import os

import numpy as np
import casadi
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation

os.makedirs('images',exist_ok=True)
if os.name == 'nt':
    plt.rcParams['font.family'] = 'MS Gothic'
elif os.name == 'posix':
    !pip install japanize-matplotlib


clear_output()

In [4]:
# 倒立振子のパラメータ
g = 9.81 #重力加速度
m0 = 1 #台車の質量
m1 = 0.2 #振子1の質量
m2 = 0.2 #振子2の質量
L1 = 1 #振子1の長さ
L2 = 0.8 #振子2の長さ

d1 = m0 + m1 + m2
d2 = (m1/2 + m1)*L1
d3 = m2*L2/2
d4 = (m1/3 + m2)*L1**2
d5 = m2*L1*L2/2
d6 = m2*L2**2/3
f1 = (m1/2 + m2)*L1*g
f2 = m2*L2*g/2

nu = 1 #制御変数の次元
nx = 6 #状態変数の次元

# コスト関数の重み
Q = casadi.diag([5,20,2.5,10,0.01,0.01])
Q_f = casadi.diag([5,20,2.5,10,0.01,0.01])
R = casadi.diag([0.1])

# 予測ホライズン等
T = 1
K = 20
dt = T/K

# 制約条件
x_lb = [-np.inf,-np.inf,-np.inf,-np.inf,-np.inf,-np.inf]
x_ub = [np.inf,np.inf,np.inf,np.inf,np.inf,np.inf]
u_lb = [-15]
u_ub = [15]

# 目標値
x_ref = casadi.DM([0,0,0,0])
u_ref = casadi.DM([0])

total = nx*(K+1) + nu*K #最適化変数の次元

In [5]:
def make_f():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)

    x0 = states[0]
    x1 = states[1]
    x2 = states[2]
    x0_dot = states[3]
    x1_dot = states[4]
    x2_dot = states[5]
    u = ctrls[0]

    sin1 = casadi.sin(x1)
    cos1 = casadi.cos(x1)
    sin2 = casadi.sin(x2)
    cos2 = casadi.cos(x2)

    D = casadi.SX.zeros(3,3)
    D[0,0] = d1
    D[0,1] = d2*cos1
    D[0,2] = d3*cos2
    D[1,0] = D[0,1]
    D[1,1] = d4
    D[1,2] = d5*(cos1*cos2 + sin1*sin2)
    D[2,0] = D[0,2]
    D[2,1] = D[1,2]
    D[2,2] = d6
    D_inv = D.inv()
    C = casadi.SX.zeros(3,3)
    C[0,1] = -d2*sin1*x1_dot
    C[0,2] = -d3*sin2*x2_dot
    C[1,2] = d5*(sin1*cos2 - cos1*sin2)*x2_dot
    C[2,1] = d5*(sin1*cos2 - cos1*sin2)*x1_dot
    G = casadi.SX.zeros("G",3)
    G[1] = -f1*sin1
    G[2] = -f2*sin2
    H = casadi.SX.zeros("H",3)
    H[0] = 1
    ddot = D_inv*(H@u - G - C@(casadi.vertcat([x0,x1,x2])))


    x0_ddot = ddot[0]
    x1_ddot = ddot[1]
    x2_ddot = ddot[2]

    states_dot = casadi.vertcat(x0_dot,x1_dot,x2_dot,x0_ddot,x1_ddot,x2_ddot)

    f = casadi.Function("f",[states,ctrls],[states_dot],['x','u'],['x_dot'])
    return f

def make_F_RK4():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)

    f = make_f()

    r1 = f(x=states,u=ctrls)["x_dot"]
    r2 = f(x=states+dt*r1/2,u=ctrls)["x_dot"]
    r3 = f(x=states+dt*r2/2,u=ctrls)["x_dot"]
    r4 = f(x=states+dt*r3,u=ctrls)["x_dot"]

    states_next = states + dt*(r1+2*r2+2*r3+r4)/6

    F_RK4 = casadi.Function("F_RK4",[states,ctrls],[states_next],["x","u"],["x_next"])
    return F_RK4

def make_integrator():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)

    f = make_f()
    ode = f(x=states, u=ctrls)["x_dot"]

    dae = {"x":states,"p":ctrls,"ode":ode}

    I = casadi.integrator("I","cvodes",dae,0,dt)
    return I

def compute_stage_cost(x,u):
    x_diff = x - x_ref
    u_diff = u - u_ref
    cost = (casadi.dot(Q@x_diff,x_diff) + casadi.dot(R@u_diff,u_diff)) / 2
    return cost

def compute_terminal_cost(x):
    x_diff = x - x_ref
    cost = casadi.dot(Q_f@x_diff,x_diff) / 2
    return cost

In [ ]:
def make_nlp():
    F_RK4 = make_F_RK4()

    U = [casadi.SX.sym(f"u_{k}",nu) for k in range(K)]
    X = [casadi.SX.sym(f"x_{k}",nx) for k in range(K+1)]
    G = []

    J = 0

    for k in range(K):
        J += compute_stage_cost(X[k],U[k]) * dt
        eq = X[k+1] - F_RK4(x=X[k],u=U[k])["x_next"]
        G.append(eq)
    J += compute_terminal_cost(X[-1])

    option = {'print_time':False,'ipopt':{'max_iter':10,'print_level':0}}
    nlp = {"x":casadi.vertcat(*X,*U),"f":J,"g":casadi.vertcat(*G)}
    S = casadi.nlpsol("S","ipopt",nlp,option)
    return S

In [ ]:
def compute_optimal_control(S,x_init,x0):
    x_init = x_init.full().ravel().tolist()
    
    lbx = x_init + x_lb*K + u_lb*K
    ubx = x_init + x_ub*K + u_ub*K
    lbg = [0]*nx*K
    ubg = [0]*nx*K

    res = S(lbx=lbx,ubx=ubx,lbg=lbg,ubg=ubg,x0=x0)
    
    offset = nx*(K+1)
    x0 = res["x"]
    u_opt = x0[offset:offset+nu]
    return u_opt, x0

In [ ]:
S = make_nlp()

In [ ]:
t_span = [0,10]
t_eval = np.arange(*t_span,dt)

x_init = casadi.DM([0,np.pi,0,0]) # 初期値
x0 = casadi.DM.zeros(total)

I = make_integrator()

X = [x_init]
U = []
x_current = x_init
for t in t_eval:
    u_opt,x0 = compute_optimal_control(S,x_current,x0)
    x_current = I(x0=x_current,p=u_opt)["xf"]
    X.append(x_current)
    U.append(u_opt)

In [ ]:
X.pop()
X = np.array(X).reshape(t_eval.size,nx)
U = np.array(U).reshape(t_eval.size,nu)

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
for k in range(nx):
    plt.plot(t_eval,X[:,k],label=f"x_{k}")
plt.legend()
plt.xlabel("Time")
plt.ylabel("State")

plt.subplot(1,2,2)
for k in range(nu):
    plt.step(t_eval,U[:,k],linestyle="--",label=f"u_{k}")
plt.legend()
plt.xlabel("Time")
plt.ylabel("Control")

plt.savefig("images/chap5_mpc.png")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
frames = np.arange(0,t_eval.size)
fps = 1 / dt

def update_figure(i):
    x_lim_min = -4
    x_lim_max = 4
    y_lim_min = -2
    y_lim_max = 2
    u_scale = 15

    ax.cla()
    ax.set_xlim(x_lim_min, x_lim_max)
    ax.set_ylim(y_lim_min, y_lim_max)
    ax.set_aspect("equal")
    ax.set_title(f"t={t_eval[i]:0.2f}")

    x,theta,_,_ = X[i]
    u, = U[i]

    points = np.array([
        [x,x-l*np.sin(theta)],
        [0,l*np.cos(theta)]
    ])

    ax.hlines(0,x_lim_min,x_lim_max,colors="black")
    ax.scatter(*points,color="blue", s=50)
    ax.plot(*points, color='blue', lw=2)
    ax.arrow(x,0,u/u_scale,0,width=0.02,head_width=0.06,head_length=0.12,length_includes_head=False,color="green",zorder=3)

    w = 0.2
    h = 0.1
    rect = patches.Rectangle(xy=(x-w/2,-h/2), width=w, height=h,color="black")
    ax.add_patch(rect)

ani = FuncAnimation(fig, update_figure, frames=frames)